In [14]:
import urllib
import json
import pandas as pd

In [15]:
# Secret token
token = open("ACCESS_TOKEN.txt", "r").read()

In [9]:
# Format a request URI for the Genius API
search_term = 'Ayla Celik'
_URL_API = "https://api.genius.com/"
_URL_SEARCH = "search?q="
querystring = _URL_API + _URL_SEARCH + urllib.parse.quote(search_term)
request = urllib.request.Request(querystring)
request.add_header("Authorization", "Bearer " + token)
request.add_header("User-Agent", "")

In [16]:
df = pd.read_csv('song_ids.txt', sep=',', names=['artist_id', 'song_id'])
df.head()

,artist_id,song_id
0,498,1806643
1,498,2985000
2,498,965955
3,498,2461261
4,498,2220711


In [8]:
# Format the search query using the 'referents' api
# Given a song id, returns all of its referents (lines that have been annotated)
# and the corresponding annotations.
song_id = 2985000
querystring = "https://api.genius.com/referents?song_id=" + str(song_id)
request = urllib.request.Request(querystring)
request.add_header("Authorization", "Bearer " + token)
request.add_header("User-Agent", "")

In [9]:
# This cell does internet stuff
# Send the request to Genius, and parse the response
response = urllib.request.urlopen(request, timeout=3)
string = response.read().decode('utf-8')
json_obj = json.loads(string)

In [10]:
# The response object is a dict with two keys
json_obj.keys()

dict_keys(['meta', 'response'])

In [11]:
json_obj

{'meta': {'status': 200}, 'response': {'referents': []}}

In [12]:
# The data we want can be found here
# For this song there are 9 lyric/annotation pairs
referents = json_obj['response']['referents']
len(referents)

0

In [65]:
# Function to recursively search the annotation dict
# and get the text, skipping blockquotes, formatting, images, etc.

def recurse(children):
    st = ''
    for child in children:
        #print(child)
        if type(child) == dict and 'tag' in child:
            if child['tag'] == 'p' or child['tag'] == 'a':
                st += recurse(child['children'])
            elif child['tag'] == 'blockquote':
                st += '[BLOCKQUOTE]'
        elif type(child) == list:
            st += recurse(child)
        elif type(child) == str:
            st += child + ' '
    return st

In [66]:
# Print each lyric along with its annotation
for ref in referents:
    print(ref['fragment'])
    print('************')
    print(recurse(ref['annotations'][0]['body']['dom']['children']))
    print()

Oh, the jealousy, the greed is the unraveling
 It's the unraveling
 And it undoes all the joy that could be
************
When she’s in a relationship or in love there’s always a jealousy that comes from the person she loves. This person is also very possessive of her thus implying the “greed”. These things make her fall out of love for this person or people. The jealousy and greed ruin the relationship where as if it weren’t there, there would more happiness in the relationship or maybe they could move farther maybe marriage or something settling down like. 

I am on a lonely road and I am traveling
 Looking for the key to set me free
************
Few songs by women at this time provided such a rich and independent narrative. In Sheila Whiteley’s  “Women and Popular Music: Sexuality, Identity and Subjectivity”  this song is seen as portraying a new type of female singer/songwriter:  [BLOCKQUOTE]

Do you want, do you want, do you want to dance with me, baby
 Do you want to take a chance

In [4]:
!grep -n '30800' artists.txt

128:30800,breakbot
